# Using Autoencoder architecture to create latent representation of 1024 time domain scans

## Proposed Approaches
1. Classic Classification on encoder-decoder latent vector  
- Train encoder-decoder on reconstruction task  
- Train FCNN to classify the pulses  
- Train CNN based Auto-encoder and vizualize latent space

2. Hybrid network approach  
- Multi-headed network to produce latent space and classify from the latent space  
- Creates a classification focussed embedding  
- Joint loss function (will need loss contributions to be tuned)  

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import MSELoss
from torchsummary import summary

from torch.utils.data import DataLoader, TensorDataset
from models.Encoder_decoder import Autoencoder, CNNAutoencoder, train_autoencoder
from models.utils import identify_device, load_thz_dataset
from Simulate import simulate_reference

from models.encoder_decoder_utils import plot_latent_space, plot_latent_space_with_labels


# Load THz pulse dataset (assumes each sample is a 1D tensor of size 1024)
data = torch.load("Synthetic_data_60k.pt", weights_only=False)
synthetic_data = data["synthetic_data"]  # [N, 1024]
num_layers = data["num_layers"]

synthetic_data.shape

torch.Size([60000, 1024])

In [ ]:
deltat = 0.0194e-12 * 4 # 4x multiple from the downsampling during dataset production
L = synthetic_data.shape[1]
t_axis = np.arange(0, L*deltat, deltat)


ref_pulse = simulate_reference(L=4*L, deltat=deltat)[:L].detach().cpu().numpy()

plt.figure(figsize=(12,4))
plt.title('Example THz Time Domain pulse')
plt.plot(t_axis*1e12, ref_pulse)
plt.plot(t_axis*1e12, synthetic_data[0])
plt.ylabel('Signal')
plt.xlabel('Time, t [ps]')
plt.show()

## FCNN Auto-encoder

In [ ]:
model_summary_FCAE = Autoencoder()
summary(model_summary_FCAE, input_size=(1, 1024))

In [ ]:
# Create DataLoader
dataset = TensorDataset(synthetic_data)
FCAE_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Model + training
model = Autoencoder(input_dim=1024, latent_dim=32)
device = identify_device()
loss = train_autoencoder(model, FCAE_loader, device, num_epochs=50, lr=1e-5, verbose='epoch')


# Plot the accuracy values
plt.plot(loss)
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.title('Training loss Progress')
plt.show()

In [ ]:
sample_index = np.random.randint(0, len(synthetic_data))

print(f'Sample Index: {sample_index}')

def test_model(index):
    real_pulse = synthetic_data[index]  
    input_pulse = real_pulse.to(device)#.unsqueeze(0)

    model.eval()
    with torch.no_grad():
        recon_pulse = model(input_pulse)

    # Bring both to CPU and remove batch dim if added
    recon_pulse_cpu = recon_pulse.squeeze(0).cpu()
    real_pulse_cpu = real_pulse.cpu()


    criterion = MSELoss()
    loss = criterion(recon_pulse_cpu, real_pulse_cpu).item()
    print(f'Loss between signals: {loss}')

    # Convert to NumPy for plotting
    recon_np = recon_pulse_cpu.numpy()
    real_np = real_pulse_cpu.numpy()

    plt.figure(figsize=(12,4))
    plt.title('Example THz Time Domain pulse')
    plt.plot(t_axis*1e12, real_np, label='Input Pulse')
    plt.plot(t_axis*1e12, recon_np, label='Reconstructed Pulse')
    plt.ylabel('Signal')
    plt.xlabel('Time, t [ps]')
    plt.legend()
    plt.show()

test_model(sample_index)

In [ ]:
## NOTE: sample index: 41212  needs checking
# Example of weak loss contributor due to low overall amplitude


test_model(41212)

In [ ]:
plot_latent_space_with_labels(model, synthetic_data, num_layers, device, 'tsne', 3000)

## CNN Auto-encoder

In [3]:
model_summary_CAE = CNNAutoencoder()
summary(model_summary_CAE, input_size=(1, 1024))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 16, 512]              80
              ReLU-2              [-1, 16, 512]               0
            Conv1d-3              [-1, 32, 256]           2,080
              ReLU-4              [-1, 32, 256]               0
            Conv1d-5              [-1, 64, 128]           8,256
              ReLU-6              [-1, 64, 128]               0
           Flatten-7                 [-1, 8192]               0
            Linear-8                   [-1, 32]         262,176
        CNNEncoder-9                   [-1, 32]               0
           Linear-10                 [-1, 8192]         270,336
        Unflatten-11              [-1, 64, 128]               0
  ConvTranspose1d-12              [-1, 32, 256]           8,224
             ReLU-13              [-1, 32, 256]               0
  ConvTranspose1d-14              [-1, 

In [4]:
CNN_loader = DataLoader(dataset, batch_size=64, shuffle=True)

model_CNNAE = CNNAutoencoder(input_dim=1024, latent_dim=32)
model_summary_CAE.to(device)
loss = train_autoencoder(model_CNNAE, CNN_loader, device, num_epochs=50, lr=1e-5, verbose='epoch')

NameError: name 'dataset' is not defined